In [51]:
import joblib
import pandas as pd
import warnings 
warnings.filterwarnings('ignore')

In [52]:
classifier = joblib.load('../../src/models/classifier_lgbm.joblib')
threshold = .51 # ВЕРОЯТНОСТНЫЙ ПОРОГ (ОБЯЗАТЕЛЬНО).

display(classifier)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('scaler', StandardScaler(),
                                                  ['requested_amount',
                                                   'pre_money_valuation',
                                                   'team_size',
                                                   'founders_experience_years',
                                                   'annual_revenue']),
                                                 ('encoder',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['startup_stage', 'industry',
                                                   'region'])])),
                ('lgbm',
                 LGBMClassifier(class_weight='balanced', colsample_bytree=0.9,
                                learning_rate=0.05, max_depth=5,
                                min_child_samples=50, n_estimators=200,
                                n_jobs=-1, objective='binary', random_state=42,
                                subsample=0.7))])

In [53]:
model = classifier.named_steps['lgbm']
prep = classifier.named_steps['preprocessor']

prep.feature_names_in_

array(['startup_stage', 'industry', 'region', 'requested_amount',
       'pre_money_valuation', 'team_size', 'founders_experience_years',
       'annual_revenue'], dtype=object)

In [54]:
import pandas as pd

startups = pd.DataFrame({
    'startup_stage': [
        'Pre-Seed',
        'Seed',
        'Series A',
        'Idea',
        'Series B',
        'Seed'
    ],
    'industry': [
        'ClimateTech',
        'FinTech',
        'HealthTech',
        'E-commerce',
        'AI',
        'EdTech'
    ],
    'region': [
        'US',
        'EU',
        'US',
        'LATAM',
        'US',
        'APAC'
    ],
    'requested_amount': [
        1_200_000,
        2_500_000,
        6_000_000,
        600_000,
        15_000_000,
        1_800_000
    ],
    'pre_money_valuation': [
        6_500_000,
        12_000_000,
        28_000_000,
        3_500_000,
        55_000_000,
        9_000_000
    ],
    'team_size': [
        7,
        10,
        14,
        3,
        18,
        6
    ],
    'founders_experience_years': [
        2,
        5,
        9,
        1,
        12,
        4
    ],
    'annual_revenue': [
        0,
        120_000,
        950_000,
        0,
        2_500_000,
        80_000
    ],
    'market_size_estimate': [
        8_000_000,
        35_000_000,
        90_000_000,
        6_000_000,
        150_000_000,
        25_000_000
    ],
    'investment_amount': [
        0,
        1_800_000,
        5_500_000,
        0,
        14_000_000,
        1_200_000
    ]
})

startups

,startup_stage,industry,region,requested_amount,pre_money_valuation,team_size,founders_experience_years,annual_revenue,market_size_estimate,investment_amount
0,Pre-Seed,ClimateTech,US,1200000,6500000,7,2,0,8000000,0
1,Seed,FinTech,EU,2500000,12000000,10,5,120000,35000000,1800000
2,Series A,HealthTech,US,6000000,28000000,14,9,950000,90000000,5500000
3,Idea,E-commerce,LATAM,600000,3500000,3,1,0,6000000,0
4,Series B,AI,US,15000000,55000000,18,12,2500000,150000000,14000000
5,Seed,EdTech,APAC,1800000,9000000,6,4,80000,25000000,1200000


In [55]:
def evaluate_startups(classifier, startups, threshold=0.5):
    """
    classifier — обученная модель с predict_proba
    startups — DataFrame со стартапами
    threshold — порог отказа
    """
    reject_proba = classifier.predict_proba(startups)[:, 1]

    results = []

    for i, proba in enumerate(reject_proba):
        decision = 'ОТКАЗАНО' if proba >= threshold else 'ПРИНЯТО'
        print(
            f"Стартап {i}: "
            f"вероятность отказа = {proba:.2f} → {decision}"
        )

        results.append({
            'startup_id': i,
            'reject_probability': proba,
            'decision': decision
        })

    return pd.DataFrame(results)

In [56]:
results = evaluate_startups(classifier, startups, threshold=threshold)
results

Стартап 0: вероятность отказа = 0.64 → ОТКАЗАНО
Стартап 1: вероятность отказа = 0.69 → ОТКАЗАНО
Стартап 2: вероятность отказа = 0.11 → ПРИНЯТО
Стартап 3: вероятность отказа = 0.93 → ОТКАЗАНО
Стартап 4: вероятность отказа = 0.12 → ПРИНЯТО
Стартап 5: вероятность отказа = 0.53 → ОТКАЗАНО


,startup_id,reject_probability,decision
0,0,0.642360,ОТКАЗАНО
1,1,0.687436,ОТКАЗАНО
2,2,0.108357,ПРИНЯТО
3,3,0.933597,ОТКАЗАНО
4,4,0.119323,ПРИНЯТО
5,5,0.529382,ОТКАЗАНО
